In [ ]:
!pip install flask
!pip install pyngrok
!pip install yt-dlp
!ngrok auth
!ngrok authtoken 2bdjDM6AHvRlEDk37SMmdIgqYjJ_3KZfGmQoUqne4A9ZAGEEo

In [ ]:
from flask import Flask, render_template, request, redirect, url_for, send_file, send_from_directory
from pyngrok import ngrok
import yt_dlp
import torchaudio
import torch
from nbconvert import NotebookExporter
from nbformat import read
from nbconvert.preprocessors import ExecutePreprocessor
import os
import threading
import shutil
import subprocess

os.environ["FLASK_ENV"] = "development"

app = Flask(__name__)

# Open an ngrok tunnel to the HTTP server
public_url = ngrok.connect(5000).public_url
print(" * ngrok tunnel \"{}\" -> \"http://127.0.0.1:{}/\"".format(public_url, 5000))

# Update any base URLs to use the public ngrok URL
app.config["BASE_URL"] = public_url

# Set up the uploads folder
uploads_folder = os.path.join(os.getcwd(), 'uploads')
os.makedirs(uploads_folder, exist_ok=True)

# Set up the downloads folder
downloads_folder = os.path.join(os.getcwd(), 'downloads')
os.makedirs(downloads_folder, exist_ok=True)

# Set up the downloads folder
records_folder = os.path.join(os.getcwd(), 'records')
os.makedirs(records_folder, exist_ok=True)

RECORDS_FOLDER = 'records'
app.config['RECORDS_FOLDER'] = RECORDS_FOLDER

# Set up the listen folder
listen_folder = os.path.join(os.getcwd(), 'listen')
os.makedirs(listen_folder, exist_ok=True)

# Set up the input folder
input_folder = os.path.join(os.getcwd(), 'input')
os.makedirs(input_folder, exist_ok=True)

# Set up the output folder
output_folder = os.path.join(os.getcwd(), 'output')
os.makedirs(output_folder, exist_ok=True)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def preprocess_notebook(file_path, notebook_path, kernel_name='python3'):
    output_path = 'output.py'

    with open(notebook_path) as f:
        nb = read(f, as_version=4)

    ep = ExecutePreprocessor(timeout=600, kernel_name=kernel_name)
    ep.preprocess(nb)

@app.route('/', methods=['GET', 'POST'])
def index():
   return render_template("index.html")

@app.route('/from-local', methods=['GET', 'POST'])
def from_local():
    uploaded_file_filename = None
    uploaded_file_name_with_ext = None

    if request.method == 'POST':
        uploaded_file = request.files['file']

        if uploaded_file.filename != '':
            file_path_with_extension = os.path.join(uploads_folder, uploaded_file.filename)
            uploaded_file.save(file_path_with_extension)

            uploaded_file_name_with_ext = uploaded_file.filename
            uploaded_file_name, _ = os.path.splitext(uploaded_file.filename)

            with app.app_context():
                app.config['UPLOADED_FILE_FILENAME'] = uploaded_file_name
                app.config['FILE_PATH_WITH_EXTENSION'] = file_path_with_extension
                app.config['UPLOADED_FILE_NAME_WITH_EXT'] = uploaded_file_name_with_ext

            track, rate = torchaudio.load(file_path_with_extension)

            preprocess_notebook(file_path_with_extension, 'separate_from_demucs.ipynb')

            uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
            file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
            uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')

    return render_template('index.html',
                       uploaded_file_filename=uploaded_file_filename,
                       uploaded_file_name_with_ext=uploaded_file_name_with_ext,
                       audioplayers_display="all_stems",
                       success_message="İşlem başarıyla tamamlandı!",
                       scroll_to_element_id="Results")

@app.route('/from-link', methods=['POST'])
def from_link():
    youtube_url = request.form['youtube_url']

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '320',
        }],
        'outtmpl': 'downloads/%(title)s.%(ext)s',
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=True)
        video_title = info_dict.get('title', None)
        video_id = info_dict.get('id', None)
        video_filename_ext = f"{info_dict['title']}.{info_dict['ext']}"
        uploaded_file_name_with_ext = video_title + '.wav'

        # İndirilen dosyayı kopyala
        source_path = os.path.join("downloads", f"{video_title}.wav")
        destination_path = os.path.join("uploads", f"{video_title}.wav")
        shutil.copy(source_path, destination_path)

        video_filename, _ = os.path.splitext(video_filename_ext)
        video_file_path = os.path.join(os.getcwd(), 'uploads', f"{video_title}.wav")

        with app.app_context():
            app.config['UPLOADED_FILE_FILENAME'] = video_filename
            app.config['FILE_PATH_WITH_EXTENSION'] = video_file_path
            app.config['UPLOADED_FILE_NAME_WITH_EXT'] = uploaded_file_name_with_ext
            app.config['YOUTUBE_VIDEO_ID'] = video_id

        preprocess_notebook(video_file_path, 'separate_from_demucs.ipynb')

        uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
        file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
        uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')
        youtube_video_id = app.config.get('YOUTUBE_VIDEO_ID')

        return render_template('index.html',
                       uploaded_file_filename=uploaded_file_filename,
                       uploaded_file_name_with_ext=uploaded_file_name_with_ext,
                       youtube_video_id = youtube_video_id,
                       audioplayers_display="all_stems",
                       success_message="İşlem başarıyla tamamlandı!",
                       scroll_to_element_id="Results")

        print(f"İndirme ve dönüştürme tamamlandı. Dosya: {destination_path}")

@app.route('/upload', methods=['POST'])
def upload():
    recorded_file_name = None
    recorded_file_name_with_ext = None

    if 'audio_data' in request.files:
        audio_file = request.files['audio_data']
        recorded_file_name = audio_file.filename
        recorded_file_name_with_ext = audio_file.filename + ".wav"
        recorded_file_path_with_extension = os.path.join(app.config['RECORDS_FOLDER'], audio_file.filename + ".wav")
        audio_file.save(recorded_file_path_with_extension)

        with app.app_context():
               app.config['RECORDED_FILE_FILENAME'] = recorded_file_name
               app.config['RECORDED_FILE_NAME_WITH_EXT'] = recorded_file_name_with_ext
               app.config['RECORDED_FILE_PATH_WITH_EXTENSION'] = recorded_file_path_with_extension
        return 'Upload successful!'
    else:
        return 'No file received.'

@app.route('/from-record', methods=['GET', 'POST'])
def from_record():

    uploaded_file_filename = app.config.get('RECORDED_FILE_FILENAME')
    file_path_with_extension = app.config.get('RECORDED_FILE_PATH_WITH_EXTENSION')
    uploaded_file_name_with_ext = app.config.get('RECORDED_FILE_NAME_WITH_EXT')

    # Kaynak ve hedef dosya yollarını belirle
    source_path = file_path_with_extension
    destination_path = os.path.join(uploads_folder)

    # Dosyayı `uploads` klasörüne kopyala
    shutil.copy(source_path, destination_path)

    with app.app_context():
            app.config['UPLOADED_FILE_FILENAME'] = uploaded_file_filename
            app.config['FILE_PATH_WITH_EXTENSION'] = file_path_with_extension
            app.config['UPLOADED_FILE_NAME_WITH_EXT'] = uploaded_file_name_with_ext

    preprocess_notebook(file_path_with_extension, 'separate_from_demucs.ipynb')

    uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
    file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
    uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')

    return render_template('index.html',
                           uploaded_file_filename=uploaded_file_filename,
                           uploaded_file_name_with_ext=uploaded_file_name_with_ext,
                           success_message="İşlem başarıyla tamamlandı!",
                           audioplayers_display="all_stems",
                           scroll_to_element_id="Results")


@app.route('/only-midi-from-local', methods=['POST'])
def only_midi_from_local():
    uploaded_file_filename = None
    uploaded_file_name_with_ext = None

    if request.method == 'POST':
        uploaded_file = request.files['file']

        if uploaded_file.filename != '':
            file_path_with_extension = os.path.join(uploads_folder, uploaded_file.filename)
            uploaded_file.save(file_path_with_extension)

            uploaded_file_name_with_ext = uploaded_file.filename
            uploaded_file_name, _ = os.path.splitext(uploaded_file.filename)

            with app.app_context():
                app.config['UPLOADED_FILE_FILENAME'] = uploaded_file_name
                app.config['FILE_PATH_WITH_EXTENSION'] = file_path_with_extension
                app.config['UPLOADED_FILE_NAME_WITH_EXT'] = uploaded_file_name_with_ext

            uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
            file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
            uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')

    return render_template('index.html',
                       uploaded_file_filename=uploaded_file_filename,
                       uploaded_file_name_with_ext=uploaded_file_name_with_ext,
                       success_message="İşlem başarıyla tamamlandı!",
                       audioplayers_display="only_midi",
                       scroll_to_element_id="Results")

@app.route('/only-midi-from-link', methods=['POST'])
def only_midi_from_link():
    youtube_url = request.form['youtube_url']

    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{
            'key': 'FFmpegExtractAudio',
            'preferredcodec': 'wav',
            'preferredquality': '320',
        }],
        'outtmpl': 'downloads/%(title)s.%(ext)s',
    }

    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        info_dict = ydl.extract_info(youtube_url, download=True)
        video_title = info_dict.get('title', None)
        video_id = info_dict.get('id', None)
        video_filename_ext = f"{info_dict['title']}.{info_dict['ext']}"
        uploaded_file_name_with_ext = video_title + '.wav'

        # İndirilen dosyayı kopyala
        source_path = os.path.join("downloads", f"{video_title}.wav")
        destination_path = os.path.join("uploads", f"{video_title}.wav")
        shutil.copy(source_path, destination_path)

        video_filename, _ = os.path.splitext(video_filename_ext)
        video_file_path = os.path.join(os.getcwd(), 'uploads', f"{video_title}.wav")

        with app.app_context():
            app.config['UPLOADED_FILE_FILENAME'] = video_filename
            app.config['FILE_PATH_WITH_EXTENSION'] = video_file_path
            app.config['UPLOADED_FILE_NAME_WITH_EXT'] = uploaded_file_name_with_ext
            app.config['YOUTUBE_VIDEO_ID'] = video_id

        uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
        file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
        uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')
        youtube_video_id = app.config.get('YOUTUBE_VIDEO_ID')

        return render_template('index.html',
                       uploaded_file_filename=uploaded_file_filename,
                       uploaded_file_name_with_ext=uploaded_file_name_with_ext,
                       youtube_video_id = youtube_video_id,
                       success_message="İşlem başarıyla tamamlandı!",
                       audioplayers_display="only_midi",
                       scroll_to_element_id="Results")

        print(f"İndirme ve dönüştürme tamamlandı. Dosya: {destination_path}")

@app.route('/only-midi-from-record', methods=['GET', 'POST'])
def only_midi_from_record():

    uploaded_file_filename = app.config.get('RECORDED_FILE_FILENAME')
    file_path_with_extension = app.config.get('RECORDED_FILE_PATH_WITH_EXTENSION')
    uploaded_file_name_with_ext = app.config.get('RECORDED_FILE_NAME_WITH_EXT')

    # Kaynak ve hedef dosya yollarını belirle
    source_path = file_path_with_extension
    destination_path = os.path.join(uploads_folder)

    # Dosyayı `uploads` klasörüne kopyala
    shutil.copy(source_path, destination_path)

    with app.app_context():
            app.config['UPLOADED_FILE_FILENAME'] = uploaded_file_filename
            app.config['FILE_PATH_WITH_EXTENSION'] = file_path_with_extension
            app.config['UPLOADED_FILE_NAME_WITH_EXT'] = uploaded_file_name_with_ext

    uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
    file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
    uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')

    return render_template('index.html',
                           uploaded_file_filename=uploaded_file_filename,
                           uploaded_file_name_with_ext=uploaded_file_name_with_ext,
                           success_message="İşlem başarıyla tamamlandı!",
                           audioplayers_display="only_midi",
                           scroll_to_element_id="Results")


@app.route('/convert-to-midi', methods=['POST'])
def convert_to_midi():
    uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
    file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
    uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')


    # Get the selected target from the form
    target = request.form.get('target')

    if target == 'original':
      target_folder = os.path.join(input_folder)
      os.makedirs(target_folder, exist_ok=True)

      source_path = os.path.join(uploads_folder, uploaded_file_name_with_ext)
      destination_path = os.path.join(target_folder, 'audio_to_midi.wav')

    else:
         target_folder = os.path.join(input_folder)
         os.makedirs(target_folder, exist_ok=True)

         source_path = os.path.join(listen_folder, 'htdemucs_6s', uploaded_file_filename, f'{target}.wav')
         destination_path = os.path.join(target_folder, 'audio_to_midi.wav')

    shutil.copy(source_path, destination_path)

    # Rename the copied file to 'audio_to_midi.wav'
    os.rename(destination_path, os.path.join(target_folder, 'audio_to_midi.wav'))

    # Yüklenen dosya adıyla aynı adda bir klasör oluştur
    separated_midi_file_folder = os.path.join(output_folder, uploaded_file_filename)
    os.makedirs(separated_midi_file_folder, exist_ok=True)

    # Yüklenen dosya adıyla aynı adda bir klasör oluştur
    target_midi_file_folder = os.path.join(separated_midi_file_folder, target)
    os.makedirs(target_midi_file_folder, exist_ok=True)

    # Yeni bir Jupyter Notebook dosyasını preprocess_notebook fonksiyonu ile başlat
    preprocess_notebook(file_path_with_extension, 'midi_from_pop2piano.ipynb')

    # output klasöründeki dosyaların adını seçilen dosyanın adıyla değiştir
    for separated_to_midi_file in os.listdir(output_folder):
      if separated_to_midi_file.endswith(('.mid', '.wav')):
        old_path = os.path.join(output_folder, separated_to_midi_file)
        # Dosyanın adı ve uzantısını ayır
        file_name, file_extension = os.path.splitext(separated_to_midi_file)
        # Yeni dosya adını oluştur
        new_name = os.path.join(output_folder, f"{target}-{uploaded_file_filename}{file_extension}")
        os.rename(old_path, new_name)

    # output klasöründeki dosyaları oluşturulan klasöre taşı
    for output_file in os.listdir(output_folder):
      if output_file.endswith(('.mid', '.wav')):
        source_path = os.path.join(output_folder, output_file)
        destination_path = os.path.join(target_midi_file_folder, output_file)
        shutil.move(source_path, destination_path)

        uploaded_file_filename = app.config.get('UPLOADED_FILE_FILENAME')
        file_path_with_extension = app.config.get('FILE_PATH_WITH_EXTENSION')
        uploaded_file_name_with_ext = app.config.get('UPLOADED_FILE_NAME_WITH_EXT')
        selected_value = app.config.get('SELECTED_VALUE')

    if selected_value == "stems-midi":
        audioplayers_display = "all_stems"
    elif selected_value == "only-midi":
        audioplayers_display = "only_midi"

    return render_template('index.html',
                       uploaded_file_filename = uploaded_file_filename,
                       uploaded_file_name_with_ext = uploaded_file_name_with_ext,
                       success_message = "MIDI Başarıyla Oluşturuldu!",
                       audioplayers_display = audioplayers_display,
                       target = target,
                       scroll_to_element_id ="Results")

@app.route('/process_selected_value', methods=['POST'])
def process_selected_value():
    data = request.get_json()
    selected_value = data.get('selected_value')

    with app.app_context():
          app.config['SELECTED_VALUE'] = selected_value

    response_data = {'message': 'İşlem başarılı. Seçilen değer: {}'.format(selected_value)}
    return 'OK!'

@app.route('/delete_recording/<filename>', methods=['GET'])
def delete_recording(filename):
    file_path = os.path.join(app.config['RECORDS_FOLDER'], filename)
    try:
        os.remove(file_path)
        return 'Delete successful!'
    except FileNotFoundError:
        return 'File not found.'

@app.route('/records/<filename>')
def listen_recorded_file(filename):
    return send_from_directory(app.config['RECORDS_FOLDER'], filename)

@app.route('/uploads/<uploaded_file_name_with_ext>')
def listen_original(uploaded_file_name_with_ext):
    file_path = os.path.join(os.getcwd(), 'uploads', uploaded_file_name_with_ext)
    return send_file(file_path, as_attachment=True)

@app.route('/listen/htdemucs_6s/<uploaded_file_filename>/<target>')
def listen_target(uploaded_file_filename, target):
    file_path = os.path.join(os.getcwd(), 'listen', 'htdemucs_6s', uploaded_file_filename, f'{target}.wav')
    return send_file(file_path, as_attachment=True)

@app.route('/output/<uploaded_file_filename>/<target>')
def listen_midi(uploaded_file_filename, target):
    file_path = os.path.join(os.getcwd(), 'output', uploaded_file_filename, target, f'{target}-{uploaded_file_filename}.mid')
    return send_file(file_path, as_attachment=True)

# Flask sunucusunu yeni bir thread'de başlat
threading.Thread(target=app.run, kwargs={"use_reloader": False}).start()
